# Single Source Shortest Path (SSSP)

In this notebook, we will use cuGraph to compute the shortest path from a starting vertex to everyother vertex in our training dataset.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* available since rerlease 0.6
* Last Edit: 02/04/2020


RAPIDS Versions: 0.12.0   

Test Hardware

* GV100 32G, CUDA 10.0




## Introduction

Single source shortest path computes the shortest paths from the given starting vertex to all other reachable vertices. 

To compute SSSP for a graph in cuGraph we use:
**cugraph.sssp(G, source)**

Input
* __G__: cugraph.Graph object
* __source__: int, Index of the source vertex

Returns 
* __df__:  a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['distance']: The computed distance from the source vertex to this vertex
    * df['predecessor']: The predecessor vertex along this paths.  Allows paths to be recreated


## cuGraph Notice 
The current version of cuGraph has some limitations:

* Vertex IDs need to be 32-bit integers.
* Vertex IDs are expected to be contiguous integers starting from 0.

cuGraph provides the renumber function to mitigate this problem. Input vertex IDs for the renumber function can be either 32-bit or 64-bit integers, can be non-contiguous, and can start from an arbitrary number. The renumber function maps the provided input vertex IDs to 32-bit contiguous integers starting from 0. cuGraph still requires the renumbered vertex IDs to be representable in 32-bit integers. These limitations are being addressed and will be fixed soon.

### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)

This is a small graph which allows for easy visual inspection to validate results.  
__Note__: The Karate dataset starts with vertex ID 1 which the cuGraph analytics assume a zero-based starting ID.  

In [ ]:
# Import needed libraries
import cudf
import cugraph

### Read the data and adjust the vertex IDs

In [ ]:
# Test file  - using the clasic Karate club dataset.  
datafile='../data/karate-data.csv'

In [ ]:
gdf = cudf.read_csv(datafile, names=["src", "dst"], delimiter='\t', dtype=["int32", "int32"])

In [ ]:
# Need to shift the vertex IDs to start with zero rather than one (next version of cuGraph will fix this issue)
gdf["src_0"] = gdf["src"] - 1
gdf["dst_0"] = gdf["dst"] - 1

# The SSSP algorithm requires that there are weights.  Just use 1.0 here (equivalent to BFS)
gdf["data"] = 1.0

In [ ]:
gdf.head()

### Create a Graph and call SSSP

In [ ]:
# create a Graph 
G = cugraph.Graph()
G.from_cudf_edgelist(gdf, source='src_0', destination='dst_0', edge_attr='data')

In [ ]:
# Call cugraph.sssp to get the distances from vertex 1:
df = cugraph.sssp(G, 1)

In [ ]:
# Print the paths
# Not using the filterred dataframe to ensure that vertex IDs match row IDs
for i in range(len(df)) :
    
    p = cugraph.utils.get_traversed_path_list(df, i)
    print(p)    


There are a number of vertices with the same distance of 3

___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___